이 단계를 마치면 과소적합과 과대적합의 개념을 이해하고, 이러한 아이디어를 적용하여 모델을 보다 정확하게 만들 수 있습니다.

# 여러 모델을 이용하여 실험

이제 신뢰할 수 있는 모델 정확도 측정법이 생겼으므로 대안 모델들을 실험하고 어떤 모델이 가장 잘 예측하는지 확인할 수 있습니다. 그런데 어떤 대안들이 있나요?

Scikit-learn의 [문서](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)를 보면 의사 결정 트리 모델은 여러 옵션을 가집니다. 가장 중요한 옵션은 트리의 깊이를 결정하는 것입니다. 첫 번째 코스에서 배웠던 것처럼 트리의 깊이는 예측 전까지 몇 번의 분리가 일어나는지를 나타냅니다. 다음은 비교적 얕은 트리입니다.

![Depth 2 Tree](http://i.imgur.com/R3ywQsR.png)

실제로, 트리에서 10개의 분리가 일어나는 것은 드문 일이 아닙니다. 트리가 깊어질수록 데이터셋은 적은 수의 주택을 가지는 잎으로 나뉩니다. 트리에 오직 한 번의 분리만 있다면 데이터는 두 개의 그룹으로 나뉩니다. 각 그룹이 한 번씩 다시 분리된다면 네 개의 그룹을 얻습니다. 이들을 다시 분리하면 8개의 그룹이 됩니다. 매 분리마다 그룹 수가 두 배가 된다면 10번의 분리 후에는 (2<sup>10</sup>)개의 그룹이 남을 것입니다. 이는 1024개의 잎입니다.

주택들을 여러 개의 잎으로 나누면 각 잎은 적은 수의 주택을 포함하게 됩니다. 주택이 매우 적은 잎은 그들과 비슷한 주택에 대해 상당히 정확한 예측을 하겠지만, 새로운 데이터에 대해 매우 신뢰할 수 없는 예측을 할 수 있습니다 (각 예측은 단지 몇 개의 주택을 기반으로 하기 때문입니다).

이는 **과대적합**이라고 불리는 현상으로, 훈련 데이터에 대해서는 거의 완벽하게 작동하지만 검증 데이터나 새로운 데이터에 대해서는 성능이 낮습니다. 반면, 우리가 트리를 매우 얕게 만들면 주택들은 뚜렷한 그룹으로 나뉘지 않습니다.

극단적인 경우, 트리가 주택들을 오직 두 개 혹은 네 개의 그룹으로 나눈다면 각 그룹은 굉장히 다양한 주택들을 포함할 것입니다. 이로 인해 훈련 데이터에서조차 대부분의 주택에 대한 예측값이 실제에서 크게 벗어날 수 있습니다. 모델이 데이터의 차이나 규칙을 발견하지 못하여 훈련 데이터에서조차 성능이 떨어지는 경우를 **과소적합**이라고 합니다.

우리는 검증 데이터로 구하는, 새로운 데이터에 대한 정확도에 관심이 있으므로 과소적합과 과대적합 사이 최적의 지점을 찾고자 합니다. 시각적으로, 우리는 아래 그림의 (빨간색) 검증 곡선의 저점을 원합니다.

![underfitting_overfitting](http://i.imgur.com/AXSEOfI.png)

# 예제
트리의 깊이를 제어하기 위한 몇 가지 대안이 있으며, 트리의 일부 경로가 다른 경로보다 깊을 수 있습니다. 그러나 *max_leaf_nodes* 인수는 과소적합과 과대적합을 제어하는 매우 합리적인 방법을 제공합니다. 모델이 더 많은 잎을 만들수록 과소적합 영역에서 과대적합 영역으로 이동합니다.

유용한 함수들을 이용해 서로 다른 *max_leaf_nodes* 값을 가질 때의 MAE 점수를 비교할 수 있습니다.

In [1]:
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

데이터는 **train_X**, **val_X**, **train_y**, **val_y**에 로드되었습니다.

In [2]:
import pandas as pd
    
# 데이터를 불러옵니다
melbourne_file_path = '../../data/melbourne-housing-snapshot/melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path) 
# 결측값을 가지는 행을 필터링합니다
filtered_melbourne_data = melbourne_data.dropna(axis=0)
# 목표값과 특성을 선택합니다
y = filtered_melbourne_data.Price
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'BuildingArea', 
                        'YearBuilt', 'Lattitude', 'Longtitude']
X = filtered_melbourne_data[melbourne_features]

from sklearn.model_selection import train_test_split

# 특성과 목표 데이터 모두를 훈련 및 검증 데이터로 나눕니다
train_X, val_X, train_y, val_y = train_test_split(X, y,random_state=0)

for 반복문을 이용하여 서로 다른 *max_leaf_nodes* 값을 가지는 모델들의 정확도를 비교할 수 있습니다.

In [3]:
# max_leaf_nodes 값에 따른 MAE 비교
for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

Max leaf nodes: 5  		 Mean Absolute Error:  347380
Max leaf nodes: 50  		 Mean Absolute Error:  258171
Max leaf nodes: 500  		 Mean Absolute Error:  243495
Max leaf nodes: 5000  		 Mean Absolute Error:  254983


나열된 옵션 중 500이 최적의 잎 개수입니다.

---

# 결론

모델은 다음과 같은 문제를 겪을 수 있습니다:
- **과대적합**: 미래에 재발하지 않을 가짜 패턴을 포착하여 예측 정확도가 떨어집니다.
- **과소적합**: 관련된 패턴을 포착하지 못해 예측 정확도가 떨어집니다.

모델 훈련에 사용되지 않은 **검증** 데이터를 사용하여 후보 모델의 정확도를 측정합니다. 이는 많은 후보 모델을 시도하고 가장 좋은 모델을 유지할 수 있도록 합니다.